# SQL

- Author: [Jinu Cho](https://github.com/jinucho)
- Peer Review: 
- Proofread:
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/14-Chains/02-SQL.ipynb) [![Open in GitHub](https://img.shields.io/badge/Open%20in%20GitHub-181717?style=flat-square&logo=github&logoColor=white)](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/14-Chains/02-SQL.ipynb)

## Overview

This tutorial covers how to use ```create_sql_query_chain``` to generate SQL queries, execute them, and derive answers. 

Additionally, let's explore the differences in operation between this method and the SQL Agent.

### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Load SQL Database](#load-sql-database)
- [SQL generate chain](#sql-generate-chain)
- [Enhance and generate answers using the LLM](#enhance-and-generate-answers-using-the-llm)
- [Agent](#Agent)
- [Differences Between create_sql_query_chain and SQL Agent](#differences-between-create_sql_query_chain-and-sql-agent)
- [[Note]: SQLite DB Creation](#note-sqlite-db-creation)



### References
- [SQLDatabase](https://python.langchain.com/api_reference/community/utilities/langchain_community.utilities.sql_database.SQLDatabase.html#sqldatabase)
- [SQL_query_chain](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.sql_database.query.create_sql_query_chain.html)
- [SQL_agent](https://python.langchain.com/api_reference/community/agent_toolkits/langchain_community.agent_toolkits.sql.base.create_sql_agent.html)
---

## Environment Setup

Setting up your environment is the first step. See the [Environment Setup](https://wikidocs.net/257836) guide for more details.


**[Note]**

The langchain-opentutorial is a package of easy-to-use environment setup guidance, useful functions and utilities for tutorials.
Check out the  [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [4]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [5]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

You can alternatively set ```OPENAI_API_KEY``` in ```.env``` file and load it. 

[Note] This is not necessary if you've already set ```OPENAI_API_KEY``` in previous steps.

In [6]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        # "OPENAI_API_KEY": "",
        # "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "02-SQL",
    }
)

Environment variables have been set successfully.


In [7]:
# Load environment variables
# Reload any variables that need to be overwritten from the previous cell

from dotenv import load_dotenv

load_dotenv(override=True)

True

## Load SQL Database

Load and verify the sample database data.

You can also create a custom database by referring to **[[Note]: SQLite DB Creation](#note-sqlite-db-creation)** .

In [8]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_sql_query_chain
from langchain_community.utilities import SQLDatabase

# Connect to the SQLite database.
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

# Output the database dialect.
print(db.dialect)

# Output the available table names.
print(db.get_usable_table_names())

sqlite
['accounts', 'customers', 'transactions']


## SQL generate chain

[RECOMMED] Create an LLM object and generate a chain by providing the LLM and DB as parameters.

Since changing the model may cause unexpected behavior, this tutorial will proceed with **gpt-3.5-turbo** .

In [14]:
# Create an OpenAI LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Generate a chain by providing the LLM and DB as parameters.
chain = create_sql_query_chain(llm=llm, db=db,k=10)# k(for query Limit)'s default value is 5

In [15]:
# Check the default prompt
print(chain.get_prompts()[0].template)

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: 

In [16]:
chain.invoke({"question": "List the all customer names."})

'SELECT "name" FROM customers;'

### If the latest version is used?

Using the latest version of OpenAI's LLM may cause issues with the output.

In [17]:
# Create an OpenAI LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Generate a chain by providing the LLM and DB as parameters.
bad_case_chain = create_sql_query_chain(llm=llm, db=db,k=10)# k(for query Limit)'s default values is 5

Unnecessary information, such as **'SQLQuery: '** , is included in the output along with the query.

In [18]:
bad_case_chain.invoke({"question": "List the all customer names."})

'SQLQuery: SELECT "name" FROM customers LIMIT 10;'

(Optional) You can specify the prompt directly using the method below.

When writing it yourself, you can include **table_info** along with descriptive **column descriptions** for better explanation.

In [19]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. 
    Unless the user specifies in his question a specific number of examples he wishes to obtain, always limit your query to at most {top_k} results. 
    You can order the results by a relevant column to return the most interesting examples in the database.

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use the following tables:
{table_info}

Here is the description of the columns in the tables:
`cust`: customer name
`prod`: product name
`trans`: transaction date

Question: {input}
"""
).partial(dialect=db.dialect)

# model 은 gpt-3.5-turbo 를 지정
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# LLM 과 DB 를 매개변수로 입력하여 chain 을 생성합니다.
chain = create_sql_query_chain(llm, db, prompt)

Executing the chain generates queries based on the database.

In [20]:
# Execute the chain and display the results.
generated_sql_query = chain.invoke({"question": "List the all customer names."})

# Print the generated query.
print(generated_sql_query.__repr__())

'SELECT name\nFROM customers'


Let's verify if the generated query executes correctly.

In [21]:
from langchain_community.tools import QuerySQLDatabaseTool

# Create a tool to execute the generated query.
execute_query = QuerySQLDatabaseTool(db=db)

In [22]:
execute_query.invoke({"query": generated_sql_query})

"[('Altman',), ('Huang',), ('Zuckerberg',), ('Musk',), ('Hassabis',), ('Chase',)]"

In [23]:
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

# Tool
execute_query = QuerySQLDatabaseTool(db=db)

# SQL query generation chain
write_query = create_sql_query_chain(llm, db,prompt)

# Create a chain to execute the generated query.
chain = write_query | execute_query

In [24]:
# Check the execution result
chain.invoke({"question": "Retrieve Altman's email address."})

"[('Sam@example.com',)]"

## Enhance and generate answers using the LLM

Using the chain created in the previous step results in short, concise answers. This can be adjusted using an LCEL-style chain to provide more natural and detailed responses.

In [25]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Define the prompt for generating answers
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

# Create a pipeline for generating natural answers
answer = answer_prompt | llm | StrOutputParser()

# Create a chain to execute the generated query and produce an answer
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer
)

In [26]:
# Check the execution result
chain.invoke({"question": "Calculate the total of Altman's transactions."})

"The total of Altman's transactions is -965.7."

## Agent

Using an Agent, you can generate SQL queries and output the results as answers.

In [29]:
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent

# Specify the model as gpt-3.5-turbo
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Connect to the SQLite database
db = SQLDatabase.from_uri("sqlite:///data/finance.db")

# Create the Agent
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [30]:
# Check the execution result
agent_executor.invoke(
    {"input": "Calculate and compare the total transactions of Altman and Zuckerberg."}
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


accounts, customers, transactions
Invoking: `sql_db_schema` with `{'table_names': 'accounts'}`



CREATE TABLE accounts (
	account_id INTEGER, 
	customer_id INTEGER, 
	balance REAL, 
	PRIMARY KEY (account_id), 
	FOREIGN KEY(customer_id) REFERENCES customers (customer_id)
)

/*
3 rows from accounts table:
account_id	customer_id	balance
1	1	1000.5
2	2	2500.75
3	3	1500.0
*/
Invoking: `sql_db_schema` with `{'table_names': 'customers'}`



CREATE TABLE customers (
	customer_id INTEGER, 
	name TEXT, 
	age INTEGER, 
	email TEXT
)

/*
3 rows from customers table:
customer_id	name	age	email
1	Altman	40	Sam@example.com
2	Huang	62	Jensen@example.com
3	Zuckerberg	41	Mark@example.com
*/
Invoking: `sql_db_schema` with `{'table_names': 'transactions'}`



CREATE TABLE transactions (
	transaction_id INTEGER, 
	account_id INTEGER, 
	amount REAL, 
	transaction_date TEXT, 
	PRIMARY KEY (transaction_id), 
	FOREIGN KEY

{'input': 'Calculate and compare the total transactions of Altman and Zuckerberg.',
 'output': 'The total transactions for each customer are as follows:\n\n- **Altman**: -965.7\n- **Zuckerberg**: 656.64\n\nThis indicates that Altman has a net loss in transactions, while Zuckerberg has a net gain.'}

## Differences Between create_sql_query_chain and SQL Agent
1. create_sql_query_chain:
    - Translates user input into a single SQL query and executes it directly.
    - Best for simple, direct query execution.
2. SQL Agent:
    - Handles more complex workflows, involving multiple queries and reasoning steps.
    - Ideal for dynamic or multi-step tasks.
3. Summary: It is recommended to use ```create_sql_query_chain``` for simple queries, while ```SQL Agent``` is suggested for complex or iterative processes.

## [Note]: SQLite DB Creation

This is a code for creating a SQLite database.

You can customize the code below to suit your needs.

In [42]:
# import sqlite3

# connection = sqlite3.connect("data/finance.db")
# cursor = connection.cursor()

# # Create `customers` table
# cursor.execute('''
#     CREATE TABLE customers (
#         customer_id INTEGER PRI
#         MARY KEY,
#         name TEXT,
#         age INTEGER,
#         email TEXT
#     );
# ''')

# # Insert data into `customers`
# cursor.executemany('''
#     INSERT INTO customers (customer_id, name, age, email)
#     VALUES (?, ?, ?, ?);
# ''', [
#     (1, 'Altman', 40, 'Sam@example.com'),
#     (2, 'Huang', 62, 'Jensen@example.com'),
#     (3, 'Zuckerberg', 41, 'Mark@example.com'),
#     (4, 'Musk', 54, 'Elon@example.com'),
#     (5, 'Hassabis', 49, 'Demis@example.com'),
#     (6, 'Chase', 35, 'Harrison@example.com')
# ])

# # Create `accounts` table
# cursor.execute('''
#     CREATE TABLE accounts (
#         account_id INTEGER PRIMARY KEY,
#         customer_id INTEGER,
#         balance REAL,
#         FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
#     );
# ''')

# # Insert data into `accounts`
# cursor.executemany('''
#     INSERT INTO accounts (account_id, customer_id, balance)
#     VALUES (?, ?, ?);
# ''', [
#     (1, 1, 1000.5),
#     (2, 2, 2500.75),
#     (3, 3, 1500.0),
#     (4, 4, 1800.25),
#     (5, 5, 2200.6),
#     (6, 6, 1750.4)
# ])

# # Create `transactions` table
# cursor.execute('''
#     CREATE TABLE transactions (
#         transaction_id INTEGER PRIMARY KEY,
#         account_id INTEGER,
#         amount REAL,
#         transaction_date TEXT,
#         FOREIGN KEY (account_id) REFERENCES accounts(account_id)
#     );
# ''')

# # Insert data into `transactions`
# cursor.executemany('''
#     INSERT INTO transactions (transaction_id, account_id, amount, transaction_date)
#     VALUES (?, ?, ?, ?);
# ''', [
#         (1, 1, 74.79, '2024-07-13'),
#         (2, 1, -224.1, '2024-05-13'),
#         (3, 1, -128.9, '2024-01-25'),
#         (4, 1, -314.05, '2024-07-28'),
#         (5, 1, -464.0, '2024-04-21'),
#         (6, 1, -486.99, '2024-07-04'),
#         (7, 1, -318.33, '2024-02-15'),
#         (8, 1, 290.23, '2024-04-06'),
#         (9, 1, 236.57, '2024-01-01'),
#         (10, 1, 321.92, '2024-05-24'),
#         (11, 1, 95.75, '2024-01-04'),
#         (12, 1, -415.22, '2024-06-10'),
#         (13, 1, 318.15, '2024-07-21'),
#         (14, 1, 269.66, '2024-01-13'),
#         (15, 1, -386.16, '2024-04-13'),
#         (16, 1, 164.98, '2024-05-02'),
#         (17, 2, -43.13, '2024-07-03'),
#         (18, 2, -308.91, '2024-03-15'),
#         (19, 2, 347.45, '2024-01-23'),
#         (20, 2, -72.35, '2024-02-01'),
#         (21, 2, -114.54, '2024-03-30'),
#         (22, 2, 273.34, '2024-07-06'),
#         (23, 2, 458.16, '2024-06-17'),
#         (24, 2, 227.77, '2024-03-14'),
#         (25, 2, -51.55, '2024-06-18'),
#         (26, 2, 304.99, '2024-02-16'),
#         (27, 2, 176.56, '2024-03-30'),
#         (28, 2, -147.86, '2024-02-16'),
#         (29, 2, -456.95, '2024-07-30'),
#         (30, 2, 190.11, '2024-01-23'),
#         (31, 2, 222.13, '2024-02-14'),
#         (32, 2, -262.09, '2024-07-21'),
#         (33, 3, 148.49, '2024-01-09'),
#         (34, 3, -289.71, '2024-01-31'),
#         (35, 3, -150.71, '2024-05-24'),
#         (36, 3, 64.14, '2024-04-08'),
#         (37, 3, -373.0, '2024-04-17'),
#         (38, 3, 210.42, '2024-06-29'),
#         (39, 3, -425.75, '2024-05-08'),
#         (40, 3, 266.32, '2024-06-05'),
#         (41, 3, 160.99, '2024-02-21'),
#         (42, 3, -127.92, '2024-04-04'),
#         (43, 3, 457.43, '2024-04-22'),
#         (44, 3, 417.64, '2024-03-28'),
#         (45, 3, 392.0, '2024-02-19'),
#         (46, 3, 221.92, '2024-02-25'),
#         (47, 3, -417.7, '2024-02-02'),
#         (48, 3, 102.08, '2024-07-07'),
#         (49, 4, 489.96, '2024-06-03'),
#         (50, 4, 21.06, '2024-04-20'),
#         (51, 4, -278.72, '2024-02-18'),
#         (52, 4, 58.08, '2024-04-15'),
#         (53, 4, 489.45, '2024-03-19'),
#         (54, 4, -448.04, '2024-07-16'),
#         (55, 4, -272.16, '2024-06-30'),
#         (56, 4, -41.19, '2024-03-21'),
#         (57, 4, 200.61, '2024-04-10'),
#         (58, 4, 431.89, '2024-03-08'),
#         (59, 4, 236.82, '2024-01-03'),
#         (60, 4, -167.94, '2024-05-31'),
#         (61, 4, 273.09, '2024-06-10'),
#         (62, 4, 225.16, '2024-03-31'),
#         (63, 4, -110.66, '2024-05-23'),
#         (64, 4, 329.74, '2024-05-29'),
#         (65, 5, -150.66, '2024-02-19'),
#         (66, 5, -406.2, '2024-06-16'),
#         (67, 5, 360.12, '2024-05-18'),
#         (68, 5, -252.19, '2024-06-11'),
#         (69, 5, -373.15, '2024-03-12'),
#         (70, 5, -361.49, '2024-05-22'),
#         (71, 5, -135.04, '2024-04-25'),
#         (72, 5, -160.99, '2024-05-31'),
#         (73, 5, 399.02, '2024-06-23'),
#         (74, 5, -77.75, '2024-05-06'),
#         (75, 5, -443.42, '2024-03-28'),
#         (76, 5, 202.05, '2024-01-04'),
#         (77, 5, -182.5, '2024-04-12'),
#         (78, 5, -428.84, '2024-03-10'),
#         (79, 5, 311.05, '2024-01-13'),
#         (80, 5, 173.89, '2024-01-10'),
#         (81, 6, -325.71, '2024-07-15'),
#         (82, 6, 127.41, '2024-01-25'),
#         (83, 6, -203.54, '2024-05-05'),
#         (84, 6, 478.24, '2024-05-07'),
#         (85, 6, -27.91, '2024-07-21'),
#         (86, 6, 194.06, '2024-01-14'),
#         (87, 6, -224.5, '2024-06-15'),
#         (88, 6, -394.75, '2024-04-18'),
#         (89, 6, 130.54, '2024-03-26'),
#         (90, 6, 494.1, '2024-02-23'),
#         (91, 6, -136.39, '2024-05-17'),
#         (92, 6, 35.12, '2024-04-28'),
#         (93, 6, -333.55, '2024-04-25'),
#         (94, 6, 170.36, '2024-03-05'),
#         (95, 6, 345.4, '2024-03-09'),
#         (96, 6, -130.69, '2024-05-24')
# ])

# connection.commit()
# connection.close()